In [2]:
import os
import csv
import numpy as np

In [3]:
###
#LEGACY
###

# with open('./data/high_resolution_2_membrane_protein_sequences_after_2000.csv', newline='') as f:
#     reader = csv.reader(f)
#     file = list(reader)  # convert the reader to a list of rows
# membrane_pdb_ids = []
# for line in file[2:]:
#     if len(line[0])==4:
#         pdb_id = line[0]
#         membrane_pdb_ids.append(pdb_id)
#     elif len(line[0])>1:
#         pdb_id = line[0][:2]+line[0][-2:]
#         membrane_pdb_ids.append(pdb_id)
        
# with open('./data/high_resolution_2_protein_sequences_after_2000.csv', newline='') as f:
#     reader = csv.reader(f)
#     file = list(reader)  # convert the reader to a list of rows

# fasta = []
# names = []
# for line in file[2:]:
#     if len(line[0])==4:
#         pdb_id = line[0]
#     elif len(line[0])>1:
#         pdb_id = line[0][:2]+line[0][-2:]
#     if not len(line[2]) or line[0] == 'Entry ID' or line[-2] != 'polypeptide(L)' or pdb_id in membrane_pdb_ids:
#         continue
#     #line = line.split(',')
#     chains = [i.strip() for i in line[2].split(',')]
#     if len(chains)-1:
#         for c in chains:
#             fasta.append(f'>{pdb_id}_{c}')
#             names.append(f'>{pdb_id}_{c}')
#             fasta.append(line[6])
#     else:
#         fasta.append(f'>{pdb_id}_{chains[0]}')
#         names.append(f'>{pdb_id}_{chains[0]}')
#         fasta.append(line[6])

In [ ]:
path_mem_proteins = './data/high_resolution_2dot8_membrane_protein_sequences_after_2000.csv'
with open(path_mem_proteins, newline='') as f:
    reader = csv.reader(f)
    mem_file = np.array(list(reader))  # convert the reader to a list of rows
membrane_pdb_ids = mem_file[:,0]

path_all_proteins = './data/high_resolution_2dot8_protein_sequences_after_2000.csv'
with open(path_all_proteins, newline='') as f:
    reader = csv.reader(f)
    prot_file = np.array(list(reader))  # convert the reader to a list of rows

In [28]:
file = np.copy(prot_file)
fasta = []
names = []
for line in file:
    pdb_id = line[0]
    if line[-1] != 'polypeptide(L)' or pdb_id in membrane_pdb_ids:
        continue
    #line = line.split(',')
    chains = [i.strip() for i in line[1].split(',')]
    if len(chains)-1:
        for c in chains:
            fasta.append(f'>{pdb_id}_{c}')
            names.append(f'>{pdb_id}_{c}')
            fasta.append(line[2])
    else:
        fasta.append(f'>{pdb_id}_{chains[0]}')
        names.append(f'>{pdb_id}_{chains[0]}')
        fasta.append(line[2])

NameError: name 'prot_file' is not defined

In [22]:
un, ind, counts = np.unique(names, return_counts=True, return_index=True)
print(len(names), len(un))

11202 11202


In [25]:
with open('./fasta/high_resolution_2dot8_membrane_protein_sequences_after_2000.fasta', 'w') as f:
    for line in fasta:
        f.write(f'{line}\n')

In [26]:
fasta_path = './fasta/high_resolution_2dot8_membrane_protein_sequences_after_2000'
# Ensure tmpDir exists
os.makedirs('./mmseq_db/tmpDir', exist_ok=True)

# Step 1: create sequence DB
os.system(f'mmseqs createdb {fasta_path}.fasta ./mmseq_db/inputDB')

# Step 2: cluster sequences
os.system('mmseqs cluster ./mmseq_db/inputDB ./mmseq_db/clusterDB ./mmseq_db/tmpDir --min-seq-id 0.3 -c 0.8 --cov-mode 1')

# Step 3: extract representative sequences
os.system('mmseqs result2repseq ./mmseq_db/inputDB ./mmseq_db/clusterDB ./mmseq_db/repDB')

# Step 4: convert the repDB to FASTA
os.system(f'mmseqs convert2fasta ./mmseq_db/repDB {fasta_path}_representatives.fasta')

# Clear mmseq_db dir
os.system('rm mmseq_db/* -rf')

createdb ./fasta/high_resolution_2dot8_membrane_protein_sequences_after_2000.fasta ./mmseq_db/inputDB 

MMseqs Version:       	13.45111
Database type         	0
Shuffle input database	true
Createdb mode         	0
Write lookup file     	1
Offset of numeric ids 	0
Compressed            	0
Verbosity             	3

Converting sequences
[=
Time for merging to inputDB_h: 0h 0m 0s 1ms
Time for merging to inputDB: 0h 0m 0s 2ms
Database type: Aminoacid
Time for processing: 0h 0m 0s 18ms
cluster ./mmseq_db/inputDB ./mmseq_db/clusterDB ./mmseq_db/tmpDir --min-seq-id 0.3 -c 0.8 --cov-mode 1 

MMseqs Version:                     	13.45111
Substitution matrix                 	nucl:nucleotide.out,aa:blosum62.out
Seed substitution matrix            	nucl:nucleotide.out,aa:VTML80.out
Sensitivity                         	4
k-mer length                        	0
k-score                             	2147483647
Alphabet size                       	nucl:5,aa:21
Max sequence length                 	65535
M

0

In [27]:
with open(f'{fasta_path}_representatives.fasta', 'r') as f:
    pdb_chain = []
    for line in f.readlines():
        if line[0]=='>':
            pdb_chain.append(line[1:])
pdb_chain_arr = np.array([i.split('_') for i in pdb_chain])
print(len(pdb_chain_arr), len(np.unique(pdb_chain_arr[:,0])))
#np.save('mem_prot_rep_2dot8.npy', pdb_chain_arr)
np.save('prot_rep_2dot8.npy', pdb_chain_arr)

1185 1185


In [66]:
with open('./fasta/high_resolution_2_protein_sequences_after_2000_representatives.fasta', 'r') as f:
    pdb_chain = []
    for line in f.readlines():
        if line[0]=='>':
            pdb_chain.append(line[1:])
pdb_chain_arr = np.array([i.split('_') for i in pdb_chain])
print(len(pdb_chain_arr), len(np.unique(pdb_chain_arr[:,0])))
np.save('prot_rep.npy', pdb_chain_arr)

17704 16683
